<a href="https://colab.research.google.com/github/007sanjib/My-test/blob/master/Demo2_Payment_Reconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install lexnlp
!pip install panda
!pip install numpy

!pip install spacy
!pip install daterangeparser
!pip install nltk
!pip install pandas
!pip install ordereddict
!pip install nb-black


     |████████████████████████████████| 10.0 MB 5.1 MB/s 
     |████████████████████████████████| 352 kB 34.1 MB/s 
     |████████████████████████████████| 1.4 MB 56.3 MB/s 
     |████████████████████████████████| 61 kB 279 kB/s 
     |████████████████████████████████| 6.8 MB 6.5 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 14.5 MB 7.6 kB/s 
     |████████████████████████████████| 294 kB 66.2 MB/s 
     |████████████████████████████████| 719 kB 65.4 MB/s 
     |████████████████████████████████| 9.5 MB 18.9 MB/s 
     |████████████████████████████████| 24.2 MB 78 kB/s 
     |████████████████████████████████| 101 kB 9.9 MB/s 
     |████████████████████████████████| 10.1 MB 13.4 MB/s 
     |████████████████████████████████| 25.9 MB 80 kB/s 
     |████████████████████████████████| 238 kB 70.1 MB/s 
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requiremen

  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7255 sha256=2ac42fa13325cc9980900b8bec1e7c1091643174a46f049137a7c0b6cbf6a772
  Stored in directory: /root/.cache/pip/wheels/e6/3d/81/a3665ce657d35359ca337b1db2975dbe5cd281a88b8982f6b6
Successfully built panda
  Created wheel for ordereddict: filename=ordereddict-1.1-py3-none-any.whl size=3552 sha256=8693faea10502ab0cf7c3f6bc1edce1b26713c263255b793d648166490ead1b7
  Stored in directory: /root/.cache/pip/wheels/97/ca/c3/78f1fe551cb748c3dc60e50b60c87febf00a3ffebd8c4e7094
Successfully built ordereddict
     |████████████████████████████████| 141 kB 5.0 MB/s 
     |████████████████████████████████| 743 kB 38.9 MB/s 
  Created wheel for nb-black: filename=nb_black-1.0.7-py3-none-any.whl size=5298 sha256=597fb4eb0ff831c9bc7584ff86f2aeae69b10a4321c84804e10d60b96381d7d6
  Stored in directory: /root/.cache/pip/wheels/1e/b2/88/51c66d23ea5fd0d40ed50997555e15d981d92671376a9a412a
Successfully built nb-black


In [12]:
!python --version


Python 3.7.11


In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [14]:
%ls "/content/drive/MyDrive/Banking-Solutions/B2B-Payment"


Sample-1-pacs-008.xml  Sample-3-pacs-008.xml  Sample-5-pacs-008.xml
Sample-2-pacs-008.xml  Sample-4-pacs-008.xml  Sample-6-pacs-008.xml


In [15]:
import pandas as pd
import numpy as np
import json

import re

import spacy
import requests
import re
import IPython
from daterangeparser import parse
from collections import OrderedDict

import operator
import glob, os
import pandas as pd
from bs4 import BeautifulSoup
import codecs
import re
import json
import copy 
from dateutil.parser import parse
import nltk

# import lexnlp.extract.en.tokens
import lexnlp.extract.en.money
import lexnlp.extract.en.en_language_tokens
import lexnlp.extract.en.entities.nltk_re
import lexnlp.extract.en.amounts
import lexnlp.extract.en.dates
import lexnlp.extract.en.geoentities
import lexnlp.extract.en.entities
import lexnlp.extract.en.percents
import lexnlp.extract.en.pii
import lexnlp.extract.en.urls

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

import xml.etree.ElementTree as et
import os
from pathlib import Path


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [16]:


pd.set_option('display.max_colwidth', 999) # prints the DataFrame cells with full text, none of its values truncated.
pd.set_option('display.max_columns', 100) # prints upto 100 columns
pd.set_option('display.max_rows', 200) # prints upto 200 rows
pd.set_option('display.colheader_justify', 'left') # dataframe column header left justified

pd.options.display.float_format = (
    "{:,.4f}".format
)  # format upto 4 decimals of float numbers

# python code style
# %load_ext nb_black

# Class to parse untructured payement (credit transfer) infomation and transform into structured JSON


In [17]:
"""
  Following format of dates are parsed

  date_array = [
    '2018-06-29 08:15:27.243860',
    'Jun 28 2018 7:40AM',
    'Jun 28 2018 at 7:40AM',
    'September 18, 2017, 22:19:55',
    'Sun, 05/12/1999, 12:30PM',
    'Mon, 21 March, 2015',
    '2018-03-12T10:12:45Z',
    '2018-06-29 17:08:00.586525+00:00',
    '2018-06-29 17:08:00.586525+05:00',
    'Tuesday , 6th September, 2017 at 4:30pm',
    # '46373892034012' # NOT A DATE
  ]

"""

"\n  Following format of dates are parsed\n\n  date_array = [\n    '2018-06-29 08:15:27.243860',\n    'Jun 28 2018 7:40AM',\n    'Jun 28 2018 at 7:40AM',\n    'September 18, 2017, 22:19:55',\n    'Sun, 05/12/1999, 12:30PM',\n    'Mon, 21 March, 2015',\n    '2018-03-12T10:12:45Z',\n    '2018-06-29 17:08:00.586525+00:00',\n    '2018-06-29 17:08:00.586525+05:00',\n    'Tuesday , 6th September, 2017 at 4:30pm',\n    # '46373892034012' # NOT A DATE\n  ]\n\n"

In [39]:
class CreditTransferTextParserNLP:

    def __init__(self):
      self.__imp_keywords_synonyms = \
      { 
        "invoice" : ["invoice", "inv", "bill", "inv.",
                     "inv#", "Invoice#", "rmtid", "remit", "rmt"],
        "account" : ["account", "a/c", "account.", "a/c.", "act", "act."],
        "number" : ["number", "no.", "#", "num", "number:"],
        "reference" : ["reference", "ref", "ref.","ref#", 
                      "refnum", "reference.", "reference#"],
        "from" : ["from"],
        "to" : ["to"],
        "tax" : ["tax","local tax", "taxing", "taxed"],
        "fee" : ["fee","fees", "charge", "chg"],
        "discount" : ["discount", "dscnt"],
        "partial" : ["partial", "part", "partially"],
        "full" : ["full","fully"],
        # identify bank as a keyword, not organization 
        "bank" : ["bank", "depository financial institution",
                  "financial institution"],
        "credit" : ["credit", "credit note", "crdt", "cren"],
        "debit" : ["debit", "debit note", "dbit", "debn"],
        "due" : ["due", "duepyblamt", ],
        "share" : ["shared", "share", "sharing"]
      }
    # end of constructor  


    def __extract_entities_spacy(self,
                                 str_param):
        """
          Description : identify all entities
          Parameters : 
            str_param : the full string 
          Return : Dictionary of all entities
        """
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(str_param)
  
        
        # ent_dict_list = []
        # for ent in doc.ents:
        #   ent_dict = {}
        #   # print("{} : {}".format(ent.text,ent.label_))
        #   ent_dict[ent.text] = ent.label_
        #   # ent_dict["start"] = ent.start 
        #   # ent_dict["end"] = ent.end
        #   # ent_dict["span"] = doc[ent.start:ent.end]  # The matched span
        #   # print("SPACY:", ent.start  , ent.end)
        #   ent_dict_list.append(ent_dict)

        ent_dict_list = [{ent.text:ent.label_}  for ent in doc.ents ]  
            

          # len(doc.ents)
        # print("ENT list", ent_dict_list)
          # sorted_entities_spacy = dict(sorted(ent_dict.items(), 
          #                           key=lambda x: x[1], reverse=False))
                                              # key=operator.itemgetter(1)))
          # print("SPACY sorted entities", sorted_entities_spacy)
          # return sorted_entities_spacy
        return ent_dict_list


    def __identify_special_keywords(self,
                                    str_param : str, 
                                    # exception_list : list
                                    ):
      """
        Description : identify all important keywords,
                      Uses NLP to create intermediate JSON
        Parameters : 
          str_param : the full string 
        Return : Dictionary of all keywords, IDs
      """
      
      # nouns = list(lexnlp.nlp.en.tokens.get_nouns(str_param, 
      #                                             lemmatize=True))
      tokens_list = lexnlp.nlp.en.tokens.get_token_list(str_param)

      # print("|||| ",tokens_list)

      dict_keywords = {}
      # position = 0

      # nn = nouns
      # nn.append("A/C")

      # identify important keywords
      for n in tokens_list:
        # print("* match", n)
        for keyword_i in self.__imp_keywords_synonyms:
          # print( n.lower())
          if n.lower() in self.__imp_keywords_synonyms[keyword_i]:
            # print("*** found match ", n)
            position = str_param.find(n)
            dict_keywords[position] = keyword_i

        # position = position + len(n)   

      # print("$$ ", dict_keywords)                  

      # Identify ID, contains nummeric(atleast one) and char
      nouns_with_num = [i  for i in tokens_list if 
          "{}".format(i).isalpha() == False and 
          any(chr.isdigit() for chr in "{}".format(i))]
          # any(map(str.isdigit, str(i))) ]  
      # print("Nouns with numbers:", nouns_with_num)

      # No ".", Minimum length is 5, contains atleast a digit  
      nouns_with_num_no_dot = [i for i in nouns_with_num if "." not 
        in "{}".format(i) and len("{}".format(i)) >= 4 ]
      # print("ID s:", nouns_with_num_no_dot)

      id_dict = {}
      for n in nouns_with_num_no_dot: 
        id_dict[str_param.find("{}".format(n))] = "{}".format(n)

      if id_dict : 
        # print("*****", sorted(id_dict.keys()))
        dict_keywords["ID"] = id_dict


      dict_keywords

      return dict_keywords



    def __identify_all_keywords(self,
                                str_param):
      """
        Description : identify all important keywords
        Parameters : 
          str_param : the full string 
        Return : Dictionary of all keywords
      """

      # print("Inside identify_all_keywords(): Length of text =",
      #       len(str_param))  

      if (not(str_param) or 
          len(str_param) == 0 or 
          len(str_param.strip()) ==0 ): 
            return np.nan

      # Firstly identify money
      money_dict_list = []
      ls_money = list(lexnlp.extract.en.money.get_money(str_param))
      # print("Money:", ls_money)
      ## DETERMINE POSITION SO THAT RELATED ENTITY CAN BE FOUND
      for dec,cur in ls_money:
        pos = str_param.find("{}".format(dec))
        if pos <0 : # position not found 
          pos = str_param.find("{}".format(int(dec)))
          if pos <0 : # position not found 
            pos = str_param.find(cur)

        money_dict_list.append( {
                        "amount":float(dec), 
                        "currency":cur,
                        "position" : pos
                        } )
        #end of for loop

      return_dict = {"Money": money_dict_list }

      #identify contact : phone, name, ssn etc.
      ls_pii = list(lexnlp.extract.en.pii.get_pii(text=str_param))
      # print("PII", ls_pii)
      # print("PII tranformed:", [{key:val} for key,val in ls_pii])
      return_dict["Contacts"] = [{k:v} for k,v in ls_pii]

      # remove the identified text so that it wont be treated as ID 
      # or another entity
      for cont in return_dict["Contacts"]:
          
          try:
            ph = cont["us_phone"]
            if ph:
              if str_param.find(ph) >= 0 :
                str_param =  str_param.replace(ph,"",1)
              else: # phone is in format 888-888-8888
                ph = ph.replace("(","",1)
                ph = ph.replace(")","",1)
                ph = ph.replace(" ","-",1)
                # print("Transformed phone number", ph)
                str_param =  str_param.replace(ph,"",1)
          except:
            pass    

      # print("*** str_param", str_param)
      ls_urls = list(lexnlp.extract.en.urls.get_urls(text=str_param))
      return_dict["URLs"] = ls_urls
      # print("*** return_dict", return_dict["URLs"])


      # Next identify percents and rates
      # print("PERCENTS AND RATES:",
      ls_percents = list(lexnlp.extract.en.percents.get_percents(
          text=str_param))
      # print("Percents & Rates", ls_percents)
      percent_dict = [{"type":str_percent, "value":float(dec),
                      "position" : str_param.find(str_percent)} \
        for (str_percent,dec,dec_transformed) in ls_percents ] 

      return_dict["Percents_Rates"] = percent_dict 


      # Use SPACY
      entities_spacy = self.__extract_entities_spacy(str_param) 

      #Next identify date
      # date_dict = {s:key for (key,value,s,e) in 
      #         entities_spacy.items() if value == "DATE"} # no duplicates
      # # print("DATES : ", date_dict)        
      # return_dict["Dates"] = date_dict
      
      date_dict = {}
      company_dict = {}
      bank_dict = {}
      location_dict = {}
      
      for ent in entities_spacy:
        start = -1
        # length = 0
        # end = 0
        # found_entity=False
        # found_key = None
        # found_value = None
        
        for key,value in ent.items():

        #   if value in ["DATE", "ORG", "GPE"]:
        #     found_entity = True
        #     found_key = key
        #     found_value = value

            
        #   if found_entity == True:
        #      if key == "start":
        #       start = value
        #      elif key == "end":
        #       end = value   

        #    # enfd of for loop   

        # if found_entity == True:    

          if value == "DATE":
            start = str_param.find(key)
            # length = len(key)
            date_dict[start] = key
            if key.isnumeric() : #it can be a ID
            # and \
            # int(key) <1900 and int(key)>2100 : #it can be a ID
                start = -1
            # found_entity=True
            
          elif value == "ORG" :
            # currenecy are recognized as ORG by SPACY
            if key.upper() not in \
            [i["currency"] for i in return_dict["Money"]]:
              start = str_param.find(key)
              # length = len(key)
              # found_entity=True
              if "bank" in key.lower():
                bank_dict[start] = key
              else:  
                company_dict[start] = key

            start = -1 # Bank names may be useful to relate entity 
          elif value == "GPE":
            start = str_param.find(key)
            # length = len(key)
            # found_entity=True
            location_dict[start] = key

          # remove so that they wont be identified as ID, entities
          # print("str_param",str_param[:start], str_param[start+length:])
          # str_param = str_param[:start] + str_param[start+length:]

          if start >= 0:
            str_param = str_param.replace(key, " "*len(key),1)



      return_dict["Dates"] = date_dict    

      company_dict["Banks"] = bank_dict

      return_dict["Companies"] = company_dict

      return_dict["Locations"] = location_dict   

      # print("after SPACY, return_dict", return_dict)

      #Next identify numbers
      ls_decimals = list(lexnlp.extract.en.amounts.get_amounts(str_param))
      # print("All Decimals :",ls_decimals)
      return_dict["Numerics"] = [{str_param.find( 
            "{}".format(int(i)) ) :float(i)} for i in ls_decimals]

      # Next identify Companies
      # company_list = list(
      #     lexnlp.extract.en.entities.nltk_re.get_companies(example))
      # Use spacy
      # company_dict = {str_param.find(key):key for (key,value,s,e) in 
      #         entities_spacy.items() if value == "ORG" and key not
      #         in [i["currency"] for i in return_dict["Money"]]} 




      # # print("ALL COMPANIES :",company_dict)
      # bank_dict = {str_param.find(key):key for (pos,key) in 
      #             company_dict.items() if key.lower().find("bank") >= 0}   
      # # print("ALL BANKS :",bank_dict)

      # company_dict = {str_param.find(key):key for (pos,key) in 
      #             company_dict.items() if key.lower().find("bank") < 0}
      # # print("SHORTLIST ORG :",company_dict)              

      # company_dict["Banks"] = bank_dict
      # # print(company_dict)

      # # return_dict["Bank"] = bank_dict
      # return_dict["Companies"] = company_dict

      # Next identity geo locations
      # location_dict = {str_param.find(key):key for (key,value,s,e) in 
      #         entities_spacy.items() if value == "GPE"}
      # # print("Locations : ", location_dict)     
      # return_dict["Locations"] = location_dict   

      # Next identify important keywords 
      return_dict["Keywords"] = self.__identify_special_keywords(
                                                        str_param) 

      return return_dict





    def __find_locations(self,
                      position_param : int,
                      str_param : str, 
                      dict_param : dict):
      
      """
        Description:
            Search nearby geo-location enity on the left side
        Paramter:
            position_param : int = position of main entity
            str_param : str = main enity
            dict_param : dict = position and entity combination      
        returns : Dict of one entity, with prepositions
          Returns : 
            String : closet location enity on right side, upto 10 characters
                    of search lenght, returns "" if not found    
      """
      return_str= ""

      all_locations = dict_param["Locations"]

      # search right side
      for pos, val in all_locations.items():
        # print("LOC : pos",pos)
        # print("LOC : position_param",position_param)
        # print("LOC : len(str_param)",len(str_param))
        # print("LOC : str_param)",str_param)
        
        # limit the search window
        if pos > position_param and \
          (pos - position_param - len(str_param)) < 10 and \
          (pos - position_param - len(str_param)) > 0: 
          # print("LOC :", val, pos - position_param - len(str_param))
          return val

      return return_str

    def __find_preposition(self,
                      position_param : int,
                      str_param : str, 
                      dict_param : dict):
      
      """
        Description:
            Search nearby preposition entity on the left side
        Paramter:
            position_param : int = postion of main entity
            str_param : str = main enity
            dict_param : dict = position and entity combination      
        returns : 
            str : the first preposition found on the left side   
      """
      return_str= ""

      all_keywords = dict_param["Keywords"]

      try:
        all_keywords.pop("ID")
      except:
        # print("Cound not pop() ID")
        pass

      for pos, val in sorted(all_keywords.items(), 
                              key= lambda kv: kv[0], reverse=True):
        # print("Inside find_entity_of(), VALUE =", 
        #       val, "Pos:", pos)
          
        if pos < position_param : # left side
          # print("Inside find_preposition(), VALUE =", val, pos)
          for key in ["from", "to", "partial", "full"]:
            for i in self.__imp_keywords_synonyms[key]:
              if val == i: #match found
                return key


      return return_str     



    def __find_entity_of(self,
                      position_param : int,
                      str_param : str, 
                      dict_param : dict):
      """
        Description:
            Search nearby enity on the left side
        Paramter:
            position_param : int = postion of main entity
            str_param : str = main enity
            dict_param : dict = position and entity combination      
        returns : Dict of one entity, with prepositions
      """
      return_dict : dict = None

      all_keywords = dict_param["Keywords"]

      if all_keywords:

        try:
          all_keywords.pop("ID")
        except:
          # print("Cound not pop() ID")
          pass

        #Firstly search in Keywords
        for pos, value in sorted(all_keywords.items(), 
                key = lambda kv: kv[0], reverse=True):
          # print("Inside find_entity_of(), VALUE =", value)
            
          # print("Looking entity for :", str_param, 
          #       "from position", position_param)  
          # print("Inside find_entity_of(), VALUE =", value, pos)
          if pos < position_param : # left side
            if value == "invoice" and \
              "{}".format(str_param).find("/") < 0:
              return {"RmtId": str_param}
            elif value == "reference":
              return { "RefNb" : str_param}  
            elif value == "account":
              prep = self.__find_preposition(pos,
                                                  value,
                                                  dict_param) 
              if prep:
                return {
                    "account_number": str_param,
                    "preposition": prep 
                        }
              else:
                return {"account_number": str_param}     

            elif value in ["fee","tax","credit", "debit","due",  
              "discount"] and "{}".format(str_param).find("/") < 0:
              return { value : str_param }

            elif value == "bank" and "{}".format(str_param).find("/") < 0:
              
              #Next search in Banks     
              # print("## Inside find_entity_of() ", dict_param)
              banks = dict_param["Companies"]["Banks"]

              if banks:
                for pos, value in sorted(banks.items(), 
                      key = lambda kv: kv[0], reverse=True):
                  # print("Inside find_entity_of(), BANK =", value, pos)
                  if pos < position_param : # left side
                      return { "bank" : value ,
                              "id" : str_param }
        


            
      # noting found
      return return_dict


    def __relate_org(self,
                      # position_param : int,
                      #  str_param : str, 
                      dict_param : dict):

      """
        Description:
          Find all Banks and Organizations, format, 
          add loacations, preositions 
        Parameter:
          Dictionary : complete list  
        Return: 
          Dictionary : formatted Oraganizations and Banks
      """       

      if not dict_param["Companies"]:
        return None

      return_list = []

      for key, value in dict_param["Companies"].items(): 

        if key == "Banks": 
          bank_dict_list = []
          for p, v in value.items() :
            if v.find("/") < 0: # Bank name not to have / 
              bank_dict = { "bank" : v}

              prep_dict = self.__find_preposition(
                                p,v,dict_param)
              if prep_dict:
                bank_dict["preposition"] = self.__find_preposition(
                                                  p,v,dict_param)
                
              loc_dict = self.__find_locations(
                                    p,v,dict_param) 
              if loc_dict:
                bank_dict["location"] = loc_dict

              bank_dict_list.append(bank_dict)

          if bank_dict_list: 
            return_list.extend(bank_dict_list)

        else:
          if value.find("/") < 0: # Org name not to have / 
            org_dict = { "organization" : value}

            loc_dict = self.__find_locations(key,value,dict_param)
            if loc_dict:
              org_dict["location"] = loc_dict

            prep_dict = self.__find_preposition(key,value,dict_param) 
            if prep_dict:
              org_dict["preposition"] = prep_dict

            return_list.append(org_dict) 
              

      return return_list                 



    def __relate_entities(self, dict_param : dict, 
                          sentence_param : str):
      """
        Description : Arrange entities in a sentence with relation, 
                      and returns one Invoice information structured
        Parameters : 
          sentence_param : full text of one sentence
          dict_param : dictionary of keywords 
        Return : 
          dict : Dictionary of one Invoice, per sentence   
          bool : another_invoice_found 
      """
      another_invoice_found = False # tested with True

      return_dict_list = []
      # print("%%%%%%%% Inside relate_entities() %%%%%%%") 
      # print(json.dumps(dict_param, indent=4, sort_keys=False)) 
      # print("***", dict_param["Keywords"])
      keywords_dict = dict_param["Keywords"]
      remaining_dict = copy.deepcopy(dict_param)
      # print("***", keywords_dict)
      ### Look for Invoice number:
      if keywords_dict:
        for key, values in keywords_dict.items():

            # print("KEY=",key)

            if "{}".format(key) == "ID":
              # print("> ID=",values)
                for pos_id ,id in values.items():

                    # print("FIND : ", id, ", position:",pos_id)
                    found_dict = self.__find_entity_of(int(pos_id),
                                                "{}".format(id), 
                                                remaining_dict)
                    if found_dict:
                      # print("Inside relate_entities(), found_dict", 
                      #       found_dict)
                      return_dict_list.append(found_dict)
                      
                      for result_key, result_values in found_dict.items(): 
                        if result_key == "RmtId":
                          another_invoice_found = True
                          # print("$$$ Keys", remaining_dict["Keywords"].keys())
                          try: # match already found
                            remaining_dict["Keywords"].pop("invoice") 
                          except:
                            # print("Key invoice is not found") 
                            pass
                    # else:
                    #    # No related entity found

            elif "{}".format(values)=="share":  # else id ID  
                print("$$$ SHARE VAL=",values)
                # for pos_id ,id in values.items():
                # print("FIND : ", id, ", position:",pos_id)
                found_dict = self.__find_entity_of(int(key),
                                "{}".format(values), remaining_dict)
                if found_dict:
                  return_dict_list.append(found_dict)
                else:
                  return_dict_list.append({"unknown": "share"})                       
 
            else:
              pass        


      #Add amount
      for m in dict_param["Money"]:
        found_related_entity = False
        for key, val in m.items():
          if key == "position":
            found_dict = self.__find_entity_of(val,
                                            m["amount"], 
                                            dict_param)
            if found_dict:
              for result_key, result_values in found_dict.items():
                
                return_dict_list.append({"Amt" : m["amount"],
                                        "Ccy": m["currency"],
                                        "related": result_key}) 
                found_related_entity = True
                break

        if found_related_entity == False:
          return_dict_list.append(m)

      #Add URL
      for u in dict_param["URLs"]:
          return_dict_list.append({"URID": u})           

      #Add Contacts 
      # print("Contacts", dict_param["Contacts"])   
      for cont in dict_param["Contacts"]:
        for key, val in cont.items():
          if "phone" in key:
            return_dict_list.append({"PhneNb": val})
          elif "name" in key:
            return_dict_list.append({"Nm": val})
          # else:
          #   # do not add ssn etc.   


      #Add Dates
      if dict_param["Dates"]:
        # print("DATES :", dict_param["Dates"])
        for pos,dt_str in dict_param["Dates"].items():
          try:
            datetime_val = parse(dt_str)
            #Convert to String below
            datetime_val = datetime_val.strftime("%m/%d/%Y, %H:%M:%S")
            found_dict = self.__find_entity_of(pos,
                                        dt_str, 
                                        dict_param)
            if found_dict:
              for result_key, result_values in found_dict.items(): 
                  return_dict_list.append({
                    "date" : datetime_val, "related" : result_key })
                
            else: # no related entity found
                return_dict_list.append({
                  "RltdDt" : datetime_val })

          except: 
            # print("Could not parse Date", dt_str)
            pass

      #Add Percents and Rates
      if dict_param["Percents_Rates"]:
        # print("DATES :", dict_param["Dates"])
        for per in dict_param["Percents_Rates"]:
          # print("PERCENTS AND RATES",per)
          # for type_str,val,pos in per.items():

          type_str = per["type"]
          val = per["value"]
          pos = per["position"]  

          found_dict = self.__find_entity_of(pos,
                                      type_str, 
                                      dict_param)
          if found_dict:
            for result_key, result_values in found_dict.items(): 
              return_dict_list.append({
                    type_str : val, "related" : result_key })
              
          else: # no related entity found
            return_dict_list.append({type_str : val })


      org_list = self.__relate_org(dict_param)
      if org_list:
        return_dict_list.extend(org_list)   

         


      return return_dict_list, another_invoice_found

    def __preprocess_contcatinated_entities(self,str_param : str):
      """
        Description : 
        
        Parameters : 
          str_param : the full string 
        Return : 
          str : 
      """

      if str_param.find("/") < 0 or \
        str_param.find("http://") >= 0 or \
        str_param.find("https://") >= 0:
        # no / found
        return str_param

      else:
        # get all token words
        tokens_list = lexnlp.nlp.en.tokens.get_token_list(str_param)

        for word in tokens_list:
          # find the token with /
          # print("TOKEN",word)
          pos = word.find("/") 
          if pos >=0:
            str_whitespace = word.replace("/", " ")
            pos_in_str = str_param.find(word)
            
            pre_word = str_param[:pos_in_str]
            # print("Pre-Word",pre_word)
            post_word = str_param[pos_in_str:]
            # print("Post-word", post_word)
            # append the new words after the original word
            str_param =  pre_word + \
            " " + str_whitespace + " " + post_word


        return str_param      

    def structure_invoice(self,str_param : str):
      """
        Description : identify all important keywords, 
                      and returns Invoice information JSON
        Parameters : 
          str_param : the full string 
        Return : Array of Dictionaies, each on one Invoice
      """
      return_dict_list = []
      one_invoice = {}

      str_param = self.__preprocess_contcatinated_entities(
                        str_param)
      # print("Going to process : ", str_param)

      ls_sentences = list( 
      lexnlp.extract.en.entities.nltk_re.get_sentence_span_list(str_param))
      # print("ALL SENTENCES", ls_sentences)

      sentence_num = 0
      invoice_num = 0
      for strat,end,sentence in ls_sentences:
        
        keywords_dict = self.__identify_all_keywords(sentence) 
        # print("@@@",json.dumps(keywords_dict, indent=4, sort_keys=False)) 
        
        structured_dict, another_invoice_found = \
          self.__relate_entities(keywords_dict, sentence)
        # print("[] Inside structure_invoice():", structured_dict)

        if structured_dict:
          # take last invoice and extend
          if len(return_dict_list) > 0 :
              invoice_dict_list = []
              invoice_dict = {}
              try:
                invoice_dict_list = return_dict_list[ 
                  len(return_dict_list) -1 ].pop(str(invoice_num))

                invoice_dict_list.extend(structured_dict)

                return_dict_list[len(return_dict_list) -1] = \
                  {str(invoice_num) : invoice_dict_list}
              except: # Invoice number is increamented
                # print("Could not pop()",invoice_num)
                return_dict_list.append(
                    {str(invoice_num) : structured_dict})

          else:
              return_dict_list = [{invoice_num : structured_dict}]

        if another_invoice_found == True:
          invoice_num = invoice_num + 1
          # print("INVOICE # :", invoice_num)     
            

        sentence_num = sentence_num + 1
        # print("Sentence ends ",str(sentence_num), "-" * 70)


      return return_dict_list





In [40]:
# ##### TESTING

# """
   
# Env-ABC/4562/2015-09-08/DEF Electronics, London/10 mln JPY/To account 23683707994215 with AAAA Bank, London (AAAAGB2L).
 
# Scenario: An invoice with number 4562, dated 08 September 2015 from DEF Electronics, London: 10 million JPY needs to be paid to DEF Electronics account 23683707994215 with AAAA Bank, London (AAAAGB2L). 
# ABC Corporation assigns reference ABC/4562/2015-09-08 to the payment. Payment transaction charges are shared between ABC Corporation and DEF Electronics.
 
# Invoise-ABC/ABC-13679/2015-09-15/GH Semiconductor Brussels/50k EURO for act BE30001216371411 DDDD Bank, Belgium (DDDDBEBB)
 
# Scenario : An invoice with number ABC-13679, dated 15 September 2015 from GHI Semiconductors, Brussels: 500,000 EUR needs to be paid to GHI Semiconductors account BE30001216371411 with DDDD Bank, Belgium (DDDDBEBB). 
# ABC Corporation assigns reference ABC/ABC-13679/2015-09-15 to the payment. 
# The accounts receivable department of GHI Semiconductors needs to be advised when the funds have been credited to the account on telephone number +32/2/2222222. 
# GHI Semiconductors will bear all payment transaction charges.
 
# Agnst Inv - ABC/987-AC/2015-09-27/1 milin USD/chg shared

# Scenario : An invoice with number 987-AC, dated 27 September 2015 from their branch ABC Corporation, California: 1 million USD needs to be paid on the branch account 4895623 with BBBB Bank, San Francisco (BBBBUS66). 
# ABC assigns reference ABC/987-AC/2015-09-27 to the payment. Payment transaction charges are shared.

# """

# text =   "Biogenetics-Crops, Glasgow, subsidiary of Biogenetics HQ, London, has received an invoice with number SX-25T, dated 13 October 2010 from Seed Inc., Dublin: 75 thousand EUR needs to be paid to the account of Seed Inc. IE29CCCC93115212345678 held at CCCC Bank, Dublin (CCCCIE2D). Transaction charges for the payment are shared between Biogenetics and Seed Inc. The payment of all Biogenetics invoices is centralised at Biogenetics HQ. It assigns reference CROPS/SX-25T/2015-10-13 to the payment order and passes it on to AAAA Bank (AAAAGB2L) where it holds the account 46373892034012. AAAA Bank sends the payment to its correspondent BBBB Bank in Dublin (BBBBIE2D)."

# example = text + " Fees is 5%."

# test_texts = [
#               "Remit ID 999666, amt INR 350, credit to Wipro Ltd., Bangalore, phone 800-400-1234 or (860) 442-1234, dated July 06 2020.",
#               example,
#               "Inv 12345678 paid, dated Aug-15-2017 amount $234, tax $5.50, from Bank of America, to Chase Bank, New York. Inv# abc-123 due amont 500 USD, fees 1%, credit  $495. Remit URL is https:\\www.example.com",
#               "Inv ABC/4562/2015-09-08/DEF Electronics Inc., London/10 million JPY/To account 23683707994215 with AAAA Bank, London (AAAAGB2L)",
#               "Invoice/ABC-13679/2015-09-15 GH Semiconductor Brussels/50k EURO for act BE30001216371411 DDDD Bank, Belgium (DDDDBEBB00)",
#               "Agnst Inv - ABC/987-AC/2015-09-27/1 mm USD/chg shared"
#               ] 

# for t in test_texts:
#   print("// ","=" * 80)
#   print("// Testing the parsing of:'",t,"'")
#   print("// ","=" * 80)
#   parser = CreditTransferTextParserNLP()
#   # print("PRE-PROCESSING:", parser.preprocess_contcatinated_entities(t))
#   payment_dict = parser.structure_invoice(t)
#   # paymennt_dict
#   for i in payment_dict:
#     print("// One Invoice (Remit)", "-"*50)
#     print(json.dumps(i, indent=4, sort_keys=False)) 

In [41]:

  

with os.scandir(
  "/content/drive/MyDrive/Banking-Solutions/B2B-Payment") as entries:

    for entry in entries:
      print( "// XML File =", entry.path)
      with open(
        "/content/drive/MyDrive/Banking-Solutions/" + 
        entry.name[:entry.name.find(".")] + ".JSON", 
        'w') as f:

        # tree = None
        tree = et.parse(entry.path)
        root = tree.getroot()
        unstructured_text_ele = root.find(
            "./CdtTrfTxInf/RmtInf/Ustrd")
        # print(unstructured_text_ele.text)

        un_text =  unstructured_text_ele.text
        un_text = un_text.strip()
        print("// ","=" * 80)
        print("// Testing the parsing of:'",un_text,"'")
        print("// ","=" * 80)
        # f.write("// The parsing of:'" + un_text +"'\n")

        parser = CreditTransferTextParserNLP()
        # print("PRE-PROCESSING:", 
        # parser.preprocess_contcatinated_entities(t))
        payment_dict = parser.structure_invoice(un_text)
        # paymennt_dict
        count = 0
        for i in payment_dict:
          count = count + 1
          print("// One Invoice (Remit)", "-"*50)
          print(json.dumps(i, indent=4, sort_keys=False))
          if count > 1:
            f.write(","+json.dumps(i, indent=4, sort_keys=False))
          else:
            f.write(" \n["+json.dumps(i, indent=4, sort_keys=False)) 

        f.write("]")
            




// XML File = /content/drive/MyDrive/Banking-Solutions/B2B-Payment/Sample-2-pacs-008.xml
//  ================================================================================
// Testing the parsing of:' Biogenetics-Crops, Glasgow, subsidiary of Biogenetics HQ, London, has received an invoice with number SX-25T, dated 13 October 2010 from Seed Inc., Dublin: 75 thousand EUR needs to be paid to the account of Seed Inc. IE29CCCC93115212345678 held at HSBC Bank, Dublin (HSBCIE2D). Transaction charges for the payment are shared between Biogenetics and Seed Inc. The payment of all Biogenetics invoices is centralised at Biogenetics HQ. It assigns reference CROPS/SX-30T/2015-10-13 to the payment order and passes it on to TD Bank (TD00562L) where it holds the account 46373892034012. TD Bank sends the payment to its correspondent HSBC Bank in Dublin (BBBBIE2D). Fees is 5%. '
//  ================================================================================
$$$ SHARE VAL= share
// One Invoice (Re